In [43]:
import re
import json
from typing import Dict, Optional, List, Tuple
import pymorphy3

# Инициализируем морфологический анализатор
morph = pymorphy3.MorphAnalyzer()


def normalize_text(text: str) -> str:
    """
    Нормализует текст, приводя все слова к начальной форме.
    
    Args:
        text: Исходный текст
        
    Returns:
        Нормализованный текст с словами в начальной форме
    """
    words = text.split()
    normalized_words = []
    
    for word in words:
        clean_word = re.sub(r'[^\w]', '', word)
        if clean_word:
            parsed = morph.parse(clean_word)[0]
            normalized_words.append(parsed.normal_form)
        else:
            normalized_words.append(word)
    
    return ' '.join(normalized_words)


def generate_word_forms(word: str) -> set:
    """
    Генерирует все возможные формы слова (падежи, числа) через pymorphy3.
    
    Args:
        word: Слово в начальной форме
        
    Returns:
        Множество всех форм слова
    """
    parsed = morph.parse(word)[0]
    forms = {word.lower()}
    
    # Генерируем все формы слова
    for form in parsed.lexeme:
        forms.add(form.word.lower())
    
    return forms


def create_flexible_pattern(alias: str) -> str:
    """
    Создает гибкий паттерн для алиаса с учетом склонений.
    
    Args:
        alias: Алиас закона в нижнем регистре
        
    Returns:
        Регулярное выражение (строка) с учетом склонений
    """
    # Разбиваем алиас на токены (слова и не-слова)
    tokens = re.findall(r'[а-яёa-z]+|[^а-яёa-z]+', alias, re.IGNORECASE)
    
    pattern_parts = []
    
    for token in tokens:
        if re.match(r'^[а-яёa-z]+$', token, re.IGNORECASE):
            # Это слово - генерируем все формы
            word_forms = generate_word_forms(token)
            
            # Если слово длинное (>3 символов) и имеет разные формы, создаем альтернативы
            if len(word_forms) > 1 and len(token) > 3:
                # Сортируем формы по длине (от длинных к коротким) для более точного матчинга
                sorted_forms = sorted(word_forms, key=len, reverse=True)
                escaped_forms = [re.escape(form) for form in sorted_forms]
                pattern_parts.append(f"(?:{'|'.join(escaped_forms)})")
            else:
                # Короткое слово или нет форм - оставляем как есть
                pattern_parts.append(re.escape(token))
        else:
            # Это не слово (пробелы, знаки препинания) - делаем гибким
            # Пробелы могут быть опциональными или множественными
            if token.strip() == '':
                pattern_parts.append(r'\s+')
            else:
                pattern_parts.append(re.escape(token))
    
    pattern = ''.join(pattern_parts)
    
    # Добавляем границы слов в начале и конце, если нужно
    if alias and re.match(r'[\wа-яёА-ЯЁ]', alias[0]):
        pattern = r'\b' + pattern
    if alias and re.match(r'[\wа-яёА-ЯЁ]', alias[-1]):
        pattern = pattern + r'\b'
    
    return pattern


def load_law_aliases_with_morphology():
    """
    Загружает law_aliases.json и создает индекс с нормализованными формами.
    ОПТИМИЗАЦИЯ: предкомпилирует все регулярные выражения один раз.
    Учитывает склонения слов в алиасах.
    
    Returns:
        Tuple[Dict, List]: (нормализованный_алиас -> [(оригинал, law_id)], все алиасы отсортированные)
    """
    with open('law_aliases.json', 'r', encoding='utf-8') as f:
        law_aliases = json.load(f)
    
    normalized_index = {}
    all_aliases = []
    
    for law_id, aliases in law_aliases.items():
        for alias in aliases:
            alias_lower = alias.lower()
            normalized = normalize_text(alias_lower)
            
            if normalized not in normalized_index:
                normalized_index[normalized] = []
            normalized_index[normalized].append((alias_lower, law_id))
            
            # ОПТИМИЗАЦИЯ: создаем гибкий паттерн с учетом склонений
            flexible_pattern_str = create_flexible_pattern(alias_lower)
            compiled_pattern = re.compile(flexible_pattern_str, re.IGNORECASE)
            
            # Также создаем точный паттерн для быстрого матчинга (без склонений)
            escaped_alias = re.escape(alias_lower)
            if re.match(r'[\w]', alias_lower):
                exact_pattern_str = r'\b' + escaped_alias
            else:
                exact_pattern_str = escaped_alias
            if re.search(r'[\w]$', alias_lower):
                exact_pattern_str = exact_pattern_str + r'\b'
            
            exact_compiled_pattern = re.compile(exact_pattern_str, re.IGNORECASE)
            
            all_aliases.append({
                'original': alias_lower,
                'normalized': normalized,
                'law_id': law_id,
                'length': len(alias_lower),
                'word_count': len(alias_lower.split()),
                'compiled_pattern': compiled_pattern,  # Гибкий паттерн с учетом склонений
                'exact_pattern': exact_compiled_pattern  # Точный паттерн для быстрого поиска
            })
    
    all_aliases.sort(key=lambda x: x['length'], reverse=True)
    
    return normalized_index, all_aliases


def find_law_in_text(text: str, normalized_index: Dict, all_aliases: List[Dict]) -> Optional[str]:
    """
    Ищет упоминание кодекса в тексте с учетом склонений.
    
    Args:
        text: Текст для поиска
        normalized_index: Индекс нормализованных форм
        all_aliases: Список всех алиасов
        
    Returns:
        law_id или None
    """
    text_lower = text.lower()
    
    # Извлекаем номера документов из текста (например, №474, №201-рп)
    numbers_in_text = re.findall(r'№\s*(\d+(?:[-./]\S*)?)', text)
    
    # Извлекаем тип документа из текста (Указ, Распоряжение, Постановление и т.д.)
    doc_type_in_text = None
    doc_type_patterns = {
        'указ': r'указ[аеуыои]?\b',
        'распоряжен': r'распоряжен[иеюя]+\b',
        'постановлен': r'постановлен[иеюя]+\b',
        'приказ': r'приказ[аеуыои]?\b',
        'закон': r'закон[аеуыои]?\b'
    }
    
    for base_type, pattern in doc_type_patterns.items():
        if re.search(pattern, text_lower):
            doc_type_in_text = base_type
            break
    
    # Если в тексте нет номера, собираем кандидатов по ключевым словам
    best_match_without_number = None
    best_match_score = 0
    
    # Сначала пробуем прямое совпадение (быстрее и точнее)
    # ОПТИМИЗАЦИЯ: используем предкомпилированные регулярные выражения
    for alias_data in all_aliases:
        alias_lower = alias_data['original']
        
        # Если в тексте есть номер документа, проверяем совместимость с алиасом
        if numbers_in_text:
            # Извлекаем номера из алиаса
            alias_numbers = re.findall(r'№\s*(\d+(?:[-./]\S*)?)', alias_lower)
            
            # Если у алиаса есть номер, он должен совпадать с номером в тексте
            if alias_numbers:
                # Нормализуем номера (убираем точки и другие знаки препинания в конце)
                normalized_text_numbers = [num.rstrip('.,;:!?') for num in numbers_in_text]
                normalized_alias_numbers = [num.rstrip('.,;:!?') for num in alias_numbers]
                has_matching_number = any(alias_num in normalized_text_numbers for alias_num in normalized_alias_numbers)
                if not has_matching_number:
                    # Пропускаем этот алиас, так как номер не совпадает
                    continue
                else:
                    # Если номер совпадает и тип документа совпадает,
                    # проверяем наличие ключевых слов (более гибко, с учетом склонений)
                    if doc_type_in_text and doc_type_in_text in alias_lower:
                        # Извлекаем ключевые слова из алиаса (слова длиннее 3 символов, буквенные)
                        alias_words = re.findall(r'\b[а-яёa-z]{4,}\b', alias_lower)
                        if alias_words:
                            # Проверяем, что большинство ключевых слов присутствуют в тексте
                            matching_count = sum(1 for word in alias_words if word in text_lower)
                            # Если хотя бы 60% ключевых слов совпадают, считаем это совпадением
                            if matching_count >= len(alias_words) * 0.6:
                                return alias_data['law_id']
        
        # Если в тексте есть тип документа, проверяем совместимость с алиасом
        if doc_type_in_text:
            # Проверяем, что алиас содержит тот же тип документа
            if doc_type_in_text not in alias_lower:
                # Пропускаем этот алиас, так как тип документа не совпадает
                continue
            
            # Если в тексте НЕТ номера, собираем кандидатов по ключевым словам (для учета склонений)
            if not numbers_in_text:
                alias_words = re.findall(r'\b[а-яёa-z]{4,}\b', alias_lower)
                if alias_words:
                    matching_count = sum(1 for word in alias_words if word in text_lower)
                    match_ratio = matching_count / len(alias_words)
                    # Сохраняем лучший результат (с максимальным процентом совпадения)
                    if match_ratio > best_match_score and match_ratio >= 0.7:
                        best_match_score = match_ratio
                        best_match_without_number = alias_data['law_id']
        
        # Сначала пробуем точное совпадение (быстрее)
        if alias_data['exact_pattern'].search(text_lower):
            return alias_data['law_id']
        
        # Если точного совпадения нет, пробуем гибкое (с учетом склонений)
        if alias_data['compiled_pattern'].search(text_lower):
            return alias_data['law_id']
    
    # Если нашли подходящий алиас по ключевым словам (для случая без номера в тексте)
    if best_match_without_number:
        return best_match_without_number
    
    # Если прямого совпадения нет, пробуем с нормализацией
    # Разбиваем текст на фразы (последовательности слов)
    word_sequences = re.finditer(r'[а-яёА-ЯЁ\w\s]+', text_lower)
    
    for seq_match in word_sequences:
        sequence = seq_match.group()
        words = sequence.split()
        
        max_window = min(10, len(words))
        
        for window_size in range(max_window, 0, -1):
            for i in range(len(words) - window_size + 1):
                window = words[i:i + window_size]
                phrase = ' '.join(window)
                
                normalized_phrase = normalize_text(phrase)
                
                if normalized_phrase in normalized_index:
                    # Дополнительная проверка для коротких алиасов
                    matches = normalized_index[normalized_phrase]
                    
                    for original_alias, law_id in matches:
                        # Если алиас короткий (как "НК", "ГК"), требуем точного совпадения
                        if len(original_alias) <= 3:
                            # Проверяем, что фраза в тексте выглядит как аббревиатура
                            if re.search(r'\b' + re.escape(phrase.upper()) + r'\b', text.upper()):
                                return law_id
                        else:
                            return law_id
    
    return None


# Загружаем данные один раз
NORMALIZED_INDEX, ALL_ALIASES = load_law_aliases_with_morphology()


def parse_legal_reference_v2(text: str) -> List[Dict[str, Optional[str]]]:
    """
    Парсит юридический текст и извлекает все упоминания статей, пунктов и подпунктов.
    Версия с поддержкой склонений через pymorphy3 и множественных ссылок.
    
    Args:
        text: Текст с упоминаниями статей, пунктов и/или подпунктов
        
    Returns:
        Список словарей с полями law_id, article, point_article, subpoint_article
    """
    results = []
    
    text_lower = text.lower().strip()
    text_stripped = text.strip()
    
    # Поиск law_id с учетом склонений
    law_id = find_law_in_text(text_lower, NORMALIZED_INDEX, ALL_ALIASES)
    
    # Паттерны для поиска статей с перечислениями
    # Ищем конструкции типа: "ст. 929, 681.14 и 1988" или "статья 100, 200 и 300"
    # Поддерживаем все склонения слова "статья": статья, статьи, статье, статью, статьёй, статьей, статей, статьям, статьями, статьях
    # Важно: "статей" (gen pl) = "стат" + "ей" (без ь), остальные формы = "стать" + окончание
    article_list_patterns = [
        r'стат(?:ь(?:ями|ях|ям|ёй|ей|[яиею])|ей)\s+((?:\d+(?:[.-]\d+)*(?:\s*,\s*|\s+и\s+|\s+или\s+)?)+)',
        r'ст\.?\s*((?:\d+(?:[.-]\d+)*(?:\s*,\s*|\s+и\s+|\s+или\s+)?)+)',
    ]
    
    # Паттерны для поиска пунктов с перечислениями
    # Разделитель: запятая (опционально с союзом) или просто союз
    point_list_patterns = [
        r'(?<!под)пункт[аеуыои]?\s+((?:(?:\d+[а-яА-Я]?|[а-яА-Я])(?:\s*,\s*(?:(?:и|или)\s+)?|\s+(?:и|или)\s+)?)+)',
        r'(?<![а-яА-Я])п\.?\s+((?:(?:\d+[а-яА-Я]?|[а-яА-Я])(?:\s*,\s*(?:(?:и|или)\s+)?|\s+(?:и|или)\s+)?)+)',
    ]
    
    # Паттерны для поиска подпунктов с перечислениями
    # Подпункты могут быть: буква (я), буква+цифры (я1), или просто цифры (26)
    # Важно: одиночная буква не должна захватываться, если за ней идёт точка (п. = пункт, а не подпункт)
    # Разделитель: запятая (опционально с союзом) или просто союз
    subpoint_list_patterns = [
        r'подпункт[аеуыои]?\s+((?:(?:[а-яА-Я](?!\.)\d*|\d+)(?:\s*,\s*(?:(?:и|или)\s+)?|\s+(?:и|или)\s+)?)+)',
        r'подп\.?\s+((?:(?:[а-яА-Я](?!\.)\d*|\d+)(?:\s*,\s*(?:(?:и|или)\s+)?|\s+(?:и|или)\s+)?)+)',
        r'пп\.?\s+((?:(?:[а-яА-Я](?!\.)\d*|\d+)(?:\s*,\s*(?:(?:и|или)\s+)?|\s+(?:и|или)\s+)?)+)',
    ]
    
    # Функция для разбора перечисления
    def parse_enumeration(enum_str: str, pattern: str) -> List[str]:
        """Разбивает строку перечисления на отдельные элементы, сохраняя регистр"""
        # Заменяем союзы на запятые для единообразия (case-insensitive)
        enum_str = re.sub(r'\s+и\s+', ',', enum_str, flags=re.IGNORECASE)
        enum_str = re.sub(r'\s+или\s+', ',', enum_str, flags=re.IGNORECASE)
        # Разбиваем по запятым
        items = [item.strip() for item in enum_str.split(',')]
        # Фильтруем пустые и оставляем только те, что соответствуют паттерну
        return [item for item in items if item and re.match(pattern, item, re.IGNORECASE)]
    
    # Поиск всех статей
    articles_found = []
    for pattern in article_list_patterns:
        matches = re.finditer(pattern, text_lower)
        for match in matches:
            # Извлекаем из оригинального текста, сохраняя регистр
            enum_str = text_stripped[match.start(1):match.end(1)]
            articles = parse_enumeration(enum_str, r'^\d+(?:[.-]\d+)*$')
            articles_found.extend(articles)
    
    # Поиск всех пунктов
    points_found = []
    for pattern in point_list_patterns:
        matches = re.finditer(pattern, text_lower)
        for match in matches:
            # Извлекаем из оригинального текста, сохраняя регистр
            enum_str = text_stripped[match.start(1):match.end(1)]
            points = parse_enumeration(enum_str, r'^(?:\d+[а-яА-Я]?|[а-яА-Я])$')
            # Фильтруем "п" - это сокращение для слова "пункт", а не номер пункта
            points = [p for p in points if p.lower() != 'п']
            points_found.extend(points)
    
    # Поиск всех подпунктов
    subpoints_found = []
    for pattern in subpoint_list_patterns:
        matches = re.finditer(pattern, text_lower)
        for match in matches:
            # Извлекаем из оригинального текста, сохраняя регистр
            enum_str = text_stripped[match.start(1):match.end(1)]
            # Подпункты: буква (я), буква+цифры (я1), или просто цифры (26)
            subpoints = parse_enumeration(enum_str, r'^(?:[а-яА-Я]\d*|\d+)$')
            # Фильтруем "п" и "подп" - это сокращения для слова "пункт/подпункт", а не номера
            subpoints = [s for s in subpoints if s.lower() not in ['п', 'подп', 'пп']]
            subpoints_found.extend(subpoints)
    
    # Конвертируем law_id из строки в число
    law_id_int = int(law_id) if law_id is not None else None
    
    # Формируем результаты на основе найденных элементов
    # Логика: создаем запись для самого детального уровня перечисления
    # Если есть несколько подпунктов -> для каждого подпункта
    # Иначе, если есть несколько пунктов -> для каждого пункта
    # Иначе, если есть несколько статей -> для каждой статьи
    
    if subpoints_found and len(subpoints_found) > 1:
        # Есть перечисление подпунктов - создаем запись для каждого
        for subpoint in subpoints_found:
            result = {
                "law_id": law_id_int,
                "article": articles_found[0] if articles_found else None,
                "point_article": points_found[0] if points_found else None,
                "subpoint_article": subpoint
            }
            results.append(result)
    elif points_found and len(points_found) > 1:
        # Есть перечисление пунктов - создаем запись для каждого
        for point in points_found:
            result = {
                "law_id": law_id_int,
                "article": articles_found[0] if articles_found else None,
                "point_article": point,
                "subpoint_article": subpoints_found[0] if subpoints_found else None
            }
            results.append(result)
    elif articles_found and len(articles_found) > 1:
        # Есть перечисление статей - создаем запись для каждой
        for article in articles_found:
            result = {
                "law_id": law_id_int,
                "article": article,
                "point_article": points_found[0] if points_found else None,
                "subpoint_article": subpoints_found[0] if subpoints_found else None
            }
            results.append(result)
    elif articles_found or points_found or subpoints_found:
        # Единичное упоминание
        result = {
            "law_id": law_id_int,
            "article": articles_found[0] if articles_found else None,
            "point_article": points_found[0] if points_found else None,
            "subpoint_article": subpoints_found[0] if subpoints_found else None
        }
        results.append(result)
    else:
        # Если ничего не найдено, возвращаем пустую запись
        results.append({
            "law_id": law_id_int,
            "article": None,
            "point_article": None,
            "subpoint_article": None
        })
    
    return results




In [44]:
# Новая функция для обработки абзацев с множественными юридическими ссылками

def find_all_law_mentions(text: str, all_aliases: List[Dict]) -> List[Dict]:
    """
    Находит все упоминания законов в тексте с их позициями.
    
    Args:
        text: Текст для поиска
        all_aliases: Список всех алиасов законов
        
    Returns:
        Список словарей с полями: law_id, start, end, matched_text
    """
    text_lower = text.lower()
    law_mentions = []
    
    # Используем все алиасы для поиска
    for alias_data in all_aliases:
        # Пробуем оба паттерна (точный и гибкий)
        for pattern in [alias_data['exact_pattern'], alias_data['compiled_pattern']]:
            for match in pattern.finditer(text_lower):
                law_mentions.append({
                    'law_id': alias_data['law_id'],
                    'start': match.start(),
                    'end': match.end(),
                    'matched_text': text[match.start():match.end()]
                })
    
    # Удаляем дубликаты и перекрывающиеся упоминания
    # Оставляем самые длинные и специфичные
    law_mentions.sort(key=lambda x: (x['start'], -(x['end'] - x['start'])))
    
    filtered_mentions = []
    for mention in law_mentions:
        # Проверяем, не перекрывается ли с уже добавленными
        overlap = False
        for existing in filtered_mentions:
            # Если упоминания перекрываются
            if not (mention['end'] <= existing['start'] or mention['start'] >= existing['end']):
                # Если текущее упоминание длиннее, заменяем
                if (mention['end'] - mention['start']) > (existing['end'] - existing['start']):
                    filtered_mentions.remove(existing)
                    break
                else:
                    overlap = True
                    break
        
        if not overlap:
            filtered_mentions.append(mention)
    
    # Сортируем по позиции в тексте
    filtered_mentions.sort(key=lambda x: x['start'])
    
    return filtered_mentions


def find_all_article_mentions(text: str) -> List[Dict]:
    """
    Находит все упоминания статей/пунктов/подпунктов в тексте с их позициями.
    
    Args:
        text: Текст для поиска
        
    Returns:
        Список словарей с полями: article, point, subpoint, start, end
    """
    text_lower = text.lower()
    text_stripped = text.strip()
    
    article_mentions = []
    
    # Паттерны для поиска статей
    article_patterns = [
        (r'стат(?:ь(?:ями|ях|ям|ёй|ей|[яиею])|ей)\s+(\d+(?:[.-]\d+)*)', 'full'),
        (r'ст\.?\s*(\d+(?:[.-]\d+)*)', 'abbr'),
    ]
    
    # Паттерны для поиска пунктов
    point_patterns = [
        (r'(?<!под)пункт[аеуыои]?\s+((?:\d+[а-яА-Я]?|[а-яА-Я]))', 'full'),
        (r'(?<![а-яА-Я])п\.?\s+((?:\d+[а-яА-Я]?|[а-яА-Я]))', 'abbr'),
    ]
    
    # Паттерны для поиска подпунктов
    subpoint_patterns = [
        (r'подпункт[аеуыои]?\s+((?:[а-яА-Я](?!\.)\d*|\d+))', 'full'),
        (r'подп\.?\s+((?:[а-яА-Я](?!\.)\d*|\d+))', 'abbr'),
        (r'пп\.?\s+((?:[а-яА-Я](?!\.)\d*|\d+))', 'abbr'),
    ]
    
    # Ищем все статьи
    for pattern_str, pattern_type in article_patterns:
        for match in re.finditer(pattern_str, text_lower):
            article_mentions.append({
                'type': 'article',
                'article': text_stripped[match.start(1):match.end(1)],
                'point': None,
                'subpoint': None,
                'start': match.start(),
                'end': match.end(),
                'full_match': text[match.start():match.end()]
            })
    
    # Ищем все пункты
    for pattern_str, pattern_type in point_patterns:
        for match in re.finditer(pattern_str, text_lower):
            point_val = text_stripped[match.start(1):match.end(1)]
            # Фильтруем "п" - это сокращение
            if point_val.lower() != 'п':
                article_mentions.append({
                    'type': 'point',
                    'article': None,
                    'point': point_val,
                    'subpoint': None,
                    'start': match.start(),
                    'end': match.end(),
                    'full_match': text[match.start():match.end()]
                })
    
    # Ищем все подпункты
    for pattern_str, pattern_type in subpoint_patterns:
        for match in re.finditer(pattern_str, text_lower):
            subpoint_val = text_stripped[match.start(1):match.end(1)]
            # Фильтруем сокращения
            if subpoint_val.lower() not in ['п', 'подп', 'пп']:
                article_mentions.append({
                    'type': 'subpoint',
                    'article': None,
                    'point': None,
                    'subpoint': subpoint_val,
                    'start': match.start(),
                    'end': match.end(),
                    'full_match': text[match.start():match.end()]
                })
    
    # Сортируем по позиции
    article_mentions.sort(key=lambda x: x['start'])
    
    return article_mentions


def parse_legal_reference_multi_law(text: str) -> List[Dict[str, Optional[str]]]:
    """
    Парсит юридический текст и извлекает все упоминания статей с привязкой к соответствующим законам.
    Поддерживает множественные ссылки на разные законы в одном тексте.
    
    Args:
        text: Текст с упоминаниями статей, пунктов и/или подпунктов
        
    Returns:
        Список словарей с полями law_id, article, point_article, subpoint_article
    """
    # Находим все упоминания законов
    law_mentions = find_all_law_mentions(text, ALL_ALIASES)
    
    # Находим все упоминания статей/пунктов/подпунктов
    article_mentions = find_all_article_mentions(text)
    
    # Если нет упоминаний законов, пробуем старый метод
    if not law_mentions:
        return parse_legal_reference_v2(text)
    
    # Если нет упоминаний статей, возвращаем только law_id
    if not article_mentions:
        results = []
        for law_mention in law_mentions:
            results.append({
                "law_id": int(law_mention['law_id']) if law_mention['law_id'] is not None else None,
                "article": None,
                "point_article": None,
                "subpoint_article": None
            })
        return results
    
    # Связываем статьи с ближайшими законами
    results = []
    used_articles = set()  # Отслеживаем уже использованные статьи
    
    for article_mention in article_mentions:
        # Пропускаем если уже обработали
        if article_mention['start'] in used_articles:
            continue
            
        # Находим ближайший закон
        # Сначала ищем справа от статьи (конструкция "ст. 374 НК РФ")
        closest_law = None
        min_distance = float('inf')
        
        for law_mention in law_mentions:
            # Расстояние от конца статьи до начала закона (закон справа)
            distance_right = law_mention['start'] - article_mention['end']
            
            # Если закон справа от статьи (в пределах 10 символов, с учетом пробелов)
            if 0 <= distance_right <= 10:
                if distance_right < min_distance:
                    closest_law = law_mention
                    min_distance = distance_right
        
        # Если не нашли закон справа, ищем слева (конструкция "НК РФ ст. 374")
        if closest_law is None:
            for law_mention in law_mentions:
                # Расстояние от конца закона до начала статьи (закон слева)
                distance_left = article_mention['start'] - law_mention['end']
                
                # Если закон слева от статьи (в пределах 200 символов)
                if 0 <= distance_left <= 200:
                    if distance_left < min_distance:
                        closest_law = law_mention
                        min_distance = distance_left
        
        # Если нашли закон, добавляем результат
        if closest_law:
            # Группируем статью, пункт и подпункт, если они рядом
            article_val = article_mention['article'] if article_mention['type'] == 'article' else None
            point_val = article_mention['point'] if article_mention['type'] == 'point' else None
            subpoint_val = article_mention['subpoint'] if article_mention['type'] == 'subpoint' else None
            
            # Ищем связанные пункты и подпункты рядом со статьей
            if article_val:
                # Ищем пункт перед статьей (конструкция "п. 1 ст. 374")
                for other in article_mentions:
                    if (other['type'] == 'point' and 
                        0 <= article_mention['start'] - other['end'] <= 20):
                        point_val = other['point']
                        used_articles.add(other['start'])
                        
                        # Ищем подпункт перед пунктом (конструкция "пп. 1 п. 1 ст. 374")
                        for subother in article_mentions:
                            if (subother['type'] == 'subpoint' and 
                                0 <= other['start'] - subother['end'] <= 20):
                                subpoint_val = subother['subpoint']
                                used_articles.add(subother['start'])
                                break
                        break
            
            # Если есть пункт, но нет статьи, ищем статью справа
            if point_val and not article_val:
                for other in article_mentions:
                    if (other['type'] == 'article' and 
                        0 <= other['start'] - article_mention['end'] <= 20):
                        article_val = other['article']
                        used_articles.add(other['start'])
                        break
                
                # Ищем подпункт слева от пункта
                for other in article_mentions:
                    if (other['type'] == 'subpoint' and 
                        0 <= article_mention['start'] - other['end'] <= 20):
                        subpoint_val = other['subpoint']
                        used_articles.add(other['start'])
                        break
            
            # Если есть подпункт, но нет пункта, ищем пункт справа
            if subpoint_val and not point_val:
                for other in article_mentions:
                    if (other['type'] == 'point' and 
                        0 <= other['start'] - article_mention['end'] <= 20):
                        point_val = other['point']
                        used_articles.add(other['start'])
                        
                        # Ищем статью справа от пункта
                        for subother in article_mentions:
                            if (subother['type'] == 'article' and 
                                0 <= subother['start'] - other['end'] <= 20):
                                article_val = subother['article']
                                used_articles.add(subother['start'])
                                break
                        break
            
            result = {
                "law_id": int(closest_law['law_id']) if closest_law['law_id'] is not None else None,
                "article": article_val,
                "point_article": point_val,
                "subpoint_article": subpoint_val
            }
            
            # Добавляем только если такого результата еще нет
            if result not in results:
                results.append(result)
                used_articles.add(article_mention['start'])
    
    # Если нашли законы, но не смогли связать со статьями, возвращаем хотя бы законы
    if not results and law_mentions:
        for law_mention in law_mentions:
            results.append({
                "law_id": int(law_mention['law_id']) if law_mention['law_id'] is not None else None,
                "article": None,
                "point_article": None,
                "subpoint_article": None
            })
    
    return results if results else [{"law_id": None, "article": None, "point_article": None, "subpoint_article": None}]


In [45]:
# Тестирование новой функции на примерах с множественными ссылками

test_texts = [
    # Тест 1: Два разных закона в одном предложении
    """В соответствии с пп. 1 п. 1 ст. 374 НК РФ объектами налогообложения признается недвижимое имущество, 
учитываемое на балансе организации в качестве объектов основных средств в порядке, установленном для ст. 105 УК РФ и ведения бухгалтерского учета.""",
    
    # Тест 2: Три разных закона в одном абзаце
    """Согласно статье 17 Конституции Российской Федерации, каждый гражданин имеет право на жизнь и свободу. Однако, в соответствии с п. 5 ст. 105 УК РФ, убийство двух или более лиц наказывается лишением свободы на срок до двадцати лет. Важно отметить, что согласно ст. 30 Гражданского кодекса Российской Федерации, гражданин, признанный недееспособным, не может самостоятельно заключать сделки.""",
    
    # Тест 3: Множественные статьи разных кодексов
    """В соответствии с пп. 12 п. 4 ст. 159 УК РФ, мошенничество наказывается. Согласно п. 2 ст. 228 УК РФ, незаконное хранение карается. Необходимо упомянуть и ст. 275 Налогового Кодекса Российской Федерации.""",
    
    # Тест 4: Три разных кодекса в одном абзаце
    """Согласно ст. 90 Семейного кодекса Российской Федерации, алименты могут быть взысканы в судебном порядке. Также, ст. 70 Трудового кодекса Российской Федерации определяет порядок заключения трудового договора. В соответствии с п. 1 ст. 213 Гражданского процессуального кодекса Российской Федерации, судебные решения подлежат немедленному исполнению.""",
]

print("=" * 100)
print("ТЕСТИРОВАНИЕ ФУНКЦИИ parse_legal_reference_multi_law")
print("=" * 100)

for idx, test_text in enumerate(test_texts, 1):
    print(f"\n{'='*100}")
    print(f"ТЕСТ {idx}:")
    print(f"{'='*100}")
    print(f"Текст: {test_text[:200]}...")
    print(f"\n{'-'*100}")
    
    # Получаем результаты
    results = parse_legal_reference_multi_law(test_text)
    
    print(f"\nНайдено ссылок: {len(results)}")
    print(f"\nРезультаты:")
    for i, result in enumerate(results, 1):
        print(f"{i}. law_id={result.get('law_id')}, "
              f"article={result.get('article')}, "
              f"point={result.get('point_article')}, "
              f"subpoint={result.get('subpoint_article')}")
    
    print(f"\nJSON:")
    print(json.dumps(results, indent=2, ensure_ascii=False))

print(f"\n{'='*100}")
print("ТЕСТИРОВАНИЕ ЗАВЕРШЕНО")
print(f"{'='*100}")


ТЕСТИРОВАНИЕ ФУНКЦИИ parse_legal_reference_multi_law

ТЕСТ 1:
Текст: В соответствии с пп. 1 п. 1 ст. 374 НК РФ объектами налогообложения признается недвижимое имущество, 
учитываемое на балансе организации в качестве объектов основных средств в порядке, установленном д...

----------------------------------------------------------------------------------------------------

Найдено ссылок: 2

Результаты:
1. law_id=15, article=374, point=1, subpoint=1
2. law_id=13, article=105, point=None, subpoint=None

JSON:
[
  {
    "law_id": 15,
    "article": "374",
    "point_article": "1",
    "subpoint_article": "1"
  },
  {
    "law_id": 13,
    "article": "105",
    "point_article": null,
    "subpoint_article": null
  }
]

ТЕСТ 2:
Текст: Согласно статье 17 Конституции Российской Федерации, каждый гражданин имеет право на жизнь и свободу. Однако, в соответствии с п. 5 ст. 105 УК РФ, убийство двух или более лиц наказывается лишением сво...

------------------------------------------------------

In [46]:
# Детальный анализ одного примера из README

complex_text = """В соответствии с пп. 1 п. 1 ст. 374 НК РФ объектами налогообложения признается недвижимое имущество, 
учитываемое на балансе организации в качестве объектов основных средств в порядке, установленном для ст. 105 УК РФ и ведения бухгалтерского учета, в случае, если налоговая база в отношении такого имущества определяется как его среднегодовая стоимость. Согласно п.  10 АПК для целей бухгалтерского учета по общему правилу единицей учета основных средств является инвентарный объект.

Согласно статье 17 Конституции Российской Федерации, каждый гражданин имеет право на жизнь и свободу. Однако, в соответствии с п. 5 ст. 105 УК РФ, убийство двух или более лиц наказывается лишением свободы на срок до двадцати лет. Также, п. 3 ст. 158 УК РФ предусматривает наказание за кражу, совершенную группой лиц по предварительному сговору. Важно отметить, что согласно ст. 30 Гражданского кодекса Российской Федерации, гражданин, признанный недееспособным, не может самостоятельно заключать сделки."""

print("=" * 100)
print("ДЕТАЛЬНЫЙ АНАЛИЗ СЛОЖНОГО ТЕКСТА")
print("=" * 100)
print(f"\nТекст:\n{complex_text}")
print("\n" + "=" * 100)

# Шаг 1: Находим все упоминания законов
print("\n[ШАГ 1] Поиск упоминаний законов:")
print("-" * 100)
law_mentions = find_all_law_mentions(complex_text, ALL_ALIASES)
for i, law in enumerate(law_mentions, 1):
    print(f"{i}. law_id={law['law_id']}, позиция=[{law['start']}:{law['end']}], "
          f"текст='{law['matched_text']}'")

# Шаг 2: Находим все упоминания статей/пунктов/подпунктов
print("\n[ШАГ 2] Поиск упоминаний статей/пунктов/подпунктов:")
print("-" * 100)
article_mentions = find_all_article_mentions(complex_text)
for i, article in enumerate(article_mentions, 1):
    print(f"{i}. type={article['type']}, позиция=[{article['start']}:{article['end']}], "
          f"текст='{article['full_match']}', значение={article.get(article['type'])}")

# Шаг 3: Получаем финальные результаты
print("\n[ШАГ 3] Финальные результаты (связанные ссылки):")
print("-" * 100)
results = parse_legal_reference_multi_law(complex_text)
print(f"\nВсего найдено ссылок: {len(results)}\n")

for i, result in enumerate(results, 1):
    law_name = "Неизвестно"
    if result.get('law_id'):
        # Пытаемся найти название закона
        for law in law_mentions:
            if law['law_id'] == str(result['law_id']):
                law_name = law['matched_text']
                break
    
    print(f"{i}. {law_name}")
    print(f"   law_id={result.get('law_id')}")
    print(f"   article={result.get('article')}")
    print(f"   point={result.get('point_article')}")
    print(f"   subpoint={result.get('subpoint_article')}")
    print()

print("=" * 100)
print("JSON вывод:")
print(json.dumps(results, indent=2, ensure_ascii=False))
print("=" * 100)


ДЕТАЛЬНЫЙ АНАЛИЗ СЛОЖНОГО ТЕКСТА

Текст:
В соответствии с пп. 1 п. 1 ст. 374 НК РФ объектами налогообложения признается недвижимое имущество, 
учитываемое на балансе организации в качестве объектов основных средств в порядке, установленном для ст. 105 УК РФ и ведения бухгалтерского учета, в случае, если налоговая база в отношении такого имущества определяется как его среднегодовая стоимость. Согласно п.  10 АПК для целей бухгалтерского учета по общему правилу единицей учета основных средств является инвентарный объект.

Согласно статье 17 Конституции Российской Федерации, каждый гражданин имеет право на жизнь и свободу. Однако, в соответствии с п. 5 ст. 105 УК РФ, убийство двух или более лиц наказывается лишением свободы на срок до двадцати лет. Также, п. 3 ст. 158 УК РФ предусматривает наказание за кражу, совершенную группой лиц по предварительному сговору. Важно отметить, что согласно ст. 30 Гражданского кодекса Российской Федерации, гражданин, признанный недееспособным, не может сам

In [5]:
# Отладка: проверяем, почему статьи не находятся

test_simple = "В соответствии с пп. 1 п. 1 ст. 374 НК РФ и ст. 105 УК РФ"

print("Тестовый текст:")
print(test_simple)
print("\n" + "="*80)

# Тестируем паттерны для статей
article_patterns = [
    (r'стат(?:ь(?:ями|ях|ям|ёй|ей|[яиею])|ей)\s+(\d+(?:[.-]\d+)*)', 'full'),
    (r'ст\\.?\s*(\d+(?:[.-]\d+)*)', 'abbr'),
]

print("\nПоиск статей:")
for pattern_str, pattern_type in article_patterns:
    print(f"\nПаттерн ({pattern_type}): {pattern_str}")
    for match in re.finditer(pattern_str, test_simple.lower()):
        print(f"  Найдено: позиция=[{match.start()}:{match.end()}], текст='{test_simple[match.start():match.end()]}', article='{test_simple[match.start(1):match.end(1)]}'")

print("\n" + "="*80)
print("\nВызов find_all_article_mentions:")
articles = find_all_article_mentions(test_simple)
for i, art in enumerate(articles, 1):
    print(f"{i}. type={art['type']}, позиция=[{art['start']}:{art['end']}], текст='{art['full_match']}', значение={art.get(art['type'])}")

print("\n" + "="*80)
print("\nВызов find_all_law_mentions:")
laws = find_all_law_mentions(test_simple, ALL_ALIASES)
for i, law in enumerate(laws, 1):
    print(f"{i}. law_id={law['law_id']}, позиция=[{law['start']}:{law['end']}], текст='{law['matched_text']}'")

print("\n" + "="*80)
print("\nВызов parse_legal_reference_multi_law:")
results = parse_legal_reference_multi_law(test_simple)
for i, res in enumerate(results, 1):
    print(f"{i}. law_id={res['law_id']}, article={res['article']}, point={res['point_article']}, subpoint={res['subpoint_article']}")


Тестовый текст:
В соответствии с пп. 1 п. 1 ст. 374 НК РФ и ст. 105 УК РФ


Поиск статей:

Паттерн (full): стат(?:ь(?:ями|ях|ям|ёй|ей|[яиею])|ей)\s+(\d+(?:[.-]\d+)*)

Паттерн (abbr): ст\\.?\s*(\d+(?:[.-]\d+)*)


Вызов find_all_article_mentions:
1. type=subpoint, позиция=[17:22], текст='пп. 1', значение=1
2. type=point, позиция=[23:27], текст='п. 1', значение=1
3. type=article, позиция=[28:35], текст='ст. 374', значение=374
4. type=article, позиция=[44:51], текст='ст. 105', значение=105


Вызов find_all_law_mentions:
1. law_id=15, позиция=[36:41], текст='НК РФ'
2. law_id=13, позиция=[52:57], текст='УК РФ'


Вызов parse_legal_reference_multi_law:
1. law_id=15, article=374, point=1, subpoint=1
2. law_id=13, article=105, point=1, subpoint=1


# Итоговая документация

## Реализованная функциональность

В notebook реализована функция **`parse_legal_reference_multi_law()`**, которая обрабатывает абзацы текста с **множественными юридическими ссылками на разные законы**.

### Основные возможности:

1. **Поиск множественных законов** - находит все упоминания законов в тексте (НК РФ, УК РФ, ГК РФ, АПК РФ и т.д.)
2. **Извлечение статей/пунктов/подпунктов** - находит все упоминания статей, пунктов и подпунктов
3. **Связывание ссылок** - правильно связывает каждую статью с соответствующим законом
4. **Поддержка склонений** - использует pymorphy3 для обработки различных форм слов

### Примеры использования:

```python
# Пример 1: Два закона в одном предложении
text1 = "В соответствии с пп. 1 п. 1 ст. 374 НК РФ и ст. 105 УК РФ..."
results1 = parse_legal_reference_multi_law(text1)
# Результат:
# [
#   {"law_id": 15, "article": "374", "point_article": "1", "subpoint_article": "1"},
#   {"law_id": 13, "article": "105", "point_article": null, "subpoint_article": null}
# ]

# Пример 2: Три разных закона в абзаце  
text2 = "Согласно ст. 90 Семейного кодекса..., ст. 70 Трудового кодекса..., п. 1 ст. 213 ГПК..."
results2 = parse_legal_reference_multi_law(text2)
# Результат:
# [
#   {"law_id": 8, "article": "90", ...},
#   {"law_id": 10, "article": "70", ...},
#   {"law_id": 6, "article": "213", "point_article": "1", ...}
# ]
```

### Функции-помощники:

1. **`find_all_law_mentions(text, all_aliases)`** - находит все упоминания законов с их позициями в тексте
2. **`find_all_article_mentions(text)`** - находит все упоминания статей/пунктов/подпунктов с их позициями
3. **`parse_legal_reference_multi_law(text)`** - главная функция, объединяющая все части

### Старая функция:

Функция **`parse_legal_reference_v2()`** продолжает работать для текстов с одной юридической ссылкой. Все существующие тесты из `demo_test_cases.json` продолжают проходить.


In [6]:
# ФИНАЛЬНЫЙ ТЕСТ: Обработка абзаца из README с множественными ссылками

readme_text = """В соответствии с пп. 1 п. 1 ст. 374 НК РФ объектами налогообложения признается недвижимое имущество, 
учитываемое на балансе организации в качестве объектов основных средств в порядке, установленном для ст. 105 УК РФ и ведения бухгалтерского учета, в случае, если налоговая база в отношении такого имущества определяется как его среднегодовая стоимость. Согласно п.  10 АПК для целей бухгалтерского учета по общему правилу единицей учета основных средств является инвентарный объект."""

print("="*100)
print("ФИНАЛЬНЫЙ ТЕСТ: Обработка сложного абзаца")
print("="*100)
print(f"\nТекст:\n{readme_text}\n")
print("="*100)

# Вызываем новую функцию
results = parse_legal_reference_multi_law(readme_text)

print(f"\nНайдено юридических ссылок: {len(results)}\n")

# Выводим результаты
for i, result in enumerate(results, 1):
    law_name = ""
    law_id = result.get('law_id')
    
    # Определяем название закона
    if law_id == 15:
        law_name = "Налоговый Кодекс РФ (НК РФ)"
    elif law_id == 13:
        law_name = "Уголовный Кодекс РФ (УК РФ)"
    elif law_id == 0:
        law_name = "Арбитражный процессуальный кодекс (АПК)"
    else:
        law_name = f"Закон #{law_id}"
    
    print(f"Ссылка {i}: {law_name}")
    print(f"  law_id: {result.get('law_id')}")
    print(f"  Статья: {result.get('article') or 'не указана'}")
    print(f"  Пункт: {result.get('point_article') or 'не указан'}")
    print(f"  Подпункт: {result.get('subpoint_article') or 'не указан'}")
    print()

print("="*100)
print("JSON вывод:")
print(json.dumps(results, indent=2, ensure_ascii=False))
print("="*100)

print("\n✅ Функция успешно обработала абзац с множественными юридическими ссылками!")
print("✅ Каждая статья правильно связана с соответствующим законом!")
print("✅ Пункты и подпункты правильно извлечены!")


ФИНАЛЬНЫЙ ТЕСТ: Обработка сложного абзаца

Текст:
В соответствии с пп. 1 п. 1 ст. 374 НК РФ объектами налогообложения признается недвижимое имущество, 
учитываемое на балансе организации в качестве объектов основных средств в порядке, установленном для ст. 105 УК РФ и ведения бухгалтерского учета, в случае, если налоговая база в отношении такого имущества определяется как его среднегодовая стоимость. Согласно п.  10 АПК для целей бухгалтерского учета по общему правилу единицей учета основных средств является инвентарный объект.


Найдено юридических ссылок: 3

Ссылка 1: Налоговый Кодекс РФ (НК РФ)
  law_id: 15
  Статья: 374
  Пункт: 1
  Подпункт: 1

Ссылка 2: Уголовный Кодекс РФ (УК РФ)
  law_id: 13
  Статья: 105
  Пункт: не указан
  Подпункт: не указан

Ссылка 3: Арбитражный процессуальный кодекс (АПК)
  law_id: 0
  Статья: не указана
  Пункт: 10
  Подпункт: не указан

JSON вывод:
[
  {
    "law_id": 15,
    "article": "374",
    "point_article": "1",
    "subpoint_article": "1"
  },


In [7]:
# Загрузка и выполнение тестов из demo_test_cases.json

def run_tests():
    """
    Загружает тесты из demo_test_cases.json и прогоняет их через parse_legal_reference_v2
    """
    with open('demo_test_cases.json', 'r', encoding='utf-8') as f:
        test_cases = json.load(f)
    
    print("=" * 80)
    print("ТЕСТИРОВАНИЕ")
    print("=" * 80)
    
    total_tests = 0
    passed_tests = 0
    failed_tests = 0
    
    for idx, test_data in enumerate(test_cases, 1):
        text = test_data['text']
        expected_results = test_data['test_cases']
        
        # Получаем результаты от функции
        actual_results = parse_legal_reference_v2(text)
        
        print(f"\nТЕСТ {idx}:")
        print(f"Текст: '{text[:100]}...'")
        print()
        
        # Сравниваем результаты
        is_passed = compare_results(expected_results, actual_results)
        
        total_tests += 1
        if is_passed:
            passed_tests += 1
            print("✅ ТЕСТ ПРОЙДЕН")
        else:
            failed_tests += 1
            print("❌ ТЕСТ НЕ ПРОЙДЕН")
        
        print(f"Найдено ссылок: {len(actual_results)}")
        print("Ожидаемые результаты:")
        print(json.dumps(expected_results, indent=2, ensure_ascii=False))
        print("Полученные результаты:")
        print(json.dumps(actual_results, indent=2, ensure_ascii=False))
        print("-" * 80)
    
    print("\n" + "=" * 80)
    print(f"ИТОГО: {passed_tests}/{total_tests} тестов пройдено")
    print(f"Успешно: {passed_tests}, Провалено: {failed_tests}")
    print("=" * 80)


def compare_results(expected, actual):
    """
    Сравнивает ожидаемые и фактические результаты
    """
    if len(expected) != len(actual):
        return False
    
    # Создаем копии для сортировки
    expected_sorted = sorted(expected, key=lambda x: (
        x.get('law_id') or 0,
        x.get('article') or '',
        x.get('point_article') or '',
        x.get('subpoint_article') or ''
    ))
    
    actual_sorted = sorted(actual, key=lambda x: (
        x.get('law_id') or 0,
        x.get('article') or '',
        x.get('point_article') or '',
        x.get('subpoint_article') or ''
    ))
    
    for exp, act in zip(expected_sorted, actual_sorted):
        if (exp.get('law_id') != act.get('law_id') or
            exp.get('article') != act.get('article') or
            exp.get('point_article') != act.get('point_article') or
            exp.get('subpoint_article') != act.get('subpoint_article')):
            return False
    
    return True


def show_test_details(test_num):
    """
    Показывает детали конкретного теста
    
    Args:
        test_num: Номер теста (1-based)
    """
    with open('demo_test_cases.json', 'r', encoding='utf-8') as f:
        test_cases = json.load(f)
    
    if test_num < 1 or test_num > len(test_cases):
        print(f"Ошибка: тест {test_num} не существует. Доступны тесты 1-{len(test_cases)}")
        return
    
    test_data = test_cases[test_num - 1]
    text = test_data['text']
    expected_results = test_data['test_cases']
    
    print(f"ТЕСТ {test_num}")
    print("=" * 80)
    print(f"Текст:\n{text}\n")
    print("-" * 80)
    
    actual_results = parse_legal_reference_v2(text)
    
    print(f"\nОжидается найти: {len(expected_results)} ссылок")
    print(f"Найдено: {len(actual_results)} ссылок")
    print()
    
    print("ОЖИДАЕМЫЕ РЕЗУЛЬТАТЫ:")
    for i, exp in enumerate(expected_results, 1):
        print(f"{i}. law_id={exp.get('law_id')}, article={exp.get('article')}, "
              f"point={exp.get('point_article')}, subpoint={exp.get('subpoint_article')}")
    
    print("\nПОЛУЧЕННЫЕ РЕЗУЛЬТАТЫ:")
    for i, act in enumerate(actual_results, 1):
        print(f"{i}. law_id={act.get('law_id')}, article={act.get('article')}, "
              f"point={act.get('point_article')}, subpoint={act.get('subpoint_article')}")
    
    print("\n" + "=" * 80)
    
    # Сравнение
    is_passed = compare_results(expected_results, actual_results)
    if is_passed:
        print("✅ ТЕСТ ПРОЙДЕН")
    else:
        print("❌ ТЕСТ НЕ ПРОЙДЕН")
        print("\nРАЗЛИЧИЯ:")
        show_differences(expected_results, actual_results)


def show_differences(expected, actual):
    """
    Показывает различия между ожидаемыми и фактическими результатами
    """
    if len(expected) != len(actual):
        print(f"- Количество ссылок: ожидалось {len(expected)}, получено {len(actual)}")
    
    # Находим отличия
    for i, (exp, act) in enumerate(zip(expected, actual), 1):
        differences = []
        if exp.get('law_id') != act.get('law_id'):
            differences.append(f"law_id: {exp.get('law_id')} -> {act.get('law_id')}")
        if exp.get('article') != act.get('article'):
            differences.append(f"article: {exp.get('article')} -> {act.get('article')}")
        if exp.get('point_article') != act.get('point_article'):
            differences.append(f"point: {exp.get('point_article')} -> {act.get('point_article')}")
        if exp.get('subpoint_article') != act.get('subpoint_article'):
            differences.append(f"subpoint: {exp.get('subpoint_article')} -> {act.get('subpoint_article')}")
        
        if differences:
            print(f"Ссылка {i}: {', '.join(differences)}")


# Запуск всех тестов
run_tests()


ТЕСТИРОВАНИЕ



ТЕСТ 1:
Текст: 'Во время проверки воздушного судна была обнаружена необходимость уточнения некоторых процедур. В соо...'

✅ ТЕСТ ПРОЙДЕН
Найдено ссылок: 1
Ожидаемые результаты:
[
  {
    "law_id": 3,
    "article": "489",
    "point_article": "24",
    "subpoint_article": "и"
  }
]
Полученные результаты:
[
  {
    "law_id": 3,
    "article": "489",
    "point_article": "24",
    "subpoint_article": "и"
  }
]
--------------------------------------------------------------------------------



ТЕСТ 2:
Текст: 'В ходе заседания представитель компании-перевозчика указал на соблюдение всех требований безопасност...'

✅ ТЕСТ ПРОЙДЕН
Найдено ссылок: 5
Ожидаемые результаты:
[
  {
    "law_id": 3,
    "article": "1048.15-8",
    "point_article": "ъ",
    "subpoint_article": "я"
  },
  {
    "law_id": 3,
    "article": "1048.15-8",
    "point_article": "ъ",
    "subpoint_article": "26"
  },
  {
    "law_id": 3,
    "article": "1048.15-8",
    "point_article": "ъ",
    "subpoint_article": "29"
  },
  {
    "law_id": 3,
    "article": "1048.15-8",
    "point_article": "ъ",
    "subpoint_article": "22"
  },
  {
    "law_id": 3,
    "article": "1048.15-8",
    "point_article": "ъ",
    "subpoint_article": "3"
  }
]
Полученные результаты:
[
  {
    "law_id": 3,
    "article": "1048.15-8",
    "point_article": "ъ",
    "subpoint_article": "я"
  },
  {
    "law_id": 3,
    "article": "1048.15-8",
    "point_article": "ъ",
    "subpoint_article": "26"
  },
  {
    "law_id": 3,
    "article"


ТЕСТ 4:
Текст: 'В ходе налоговой проверки компании «ТехноТрейд» были выявлены нарушения, связанные с занижением нало...'

✅ ТЕСТ ПРОЙДЕН
Найдено ссылок: 9
Ожидаемые результаты:
[
  {
    "law_id": 15,
    "article": "158",
    "point_article": null,
    "subpoint_article": null
  },
  {
    "law_id": 15,
    "article": "1784",
    "point_article": null,
    "subpoint_article": null
  },
  {
    "law_id": 15,
    "article": "730.11",
    "point_article": null,
    "subpoint_article": null
  },
  {
    "law_id": 15,
    "article": "1722",
    "point_article": null,
    "subpoint_article": null
  },
  {
    "law_id": 15,
    "article": "249",
    "point_article": null,
    "subpoint_article": null
  },
  {
    "law_id": 15,
    "article": "51.8-7",
    "point_article": null,
    "subpoint_article": null
  },
  {
    "law_id": 15,
    "article": "1689.6-11",
    "point_article": null,
    "subpoint_article": null
  },
  {
    "law_id": 15,
    "article": "18",
    "point_article": null,
 


ТЕСТ 5:
Текст: 'Компания, занимающаяся транспортировкой грузов, была подвергнута аудиту налоговыми органами. Было ус...'

✅ ТЕСТ ПРОЙДЕН
Найдено ссылок: 4
Ожидаемые результаты:
[
  {
    "law_id": 1,
    "article": "1930.7",
    "point_article": "26",
    "subpoint_article": "Х"
  },
  {
    "law_id": 1,
    "article": "1930.7",
    "point_article": "26",
    "subpoint_article": "д"
  },
  {
    "law_id": 1,
    "article": "1930.7",
    "point_article": "26",
    "subpoint_article": "21"
  },
  {
    "law_id": 1,
    "article": "1930.7",
    "point_article": "26",
    "subpoint_article": "в"
  }
]
Полученные результаты:
[
  {
    "law_id": 1,
    "article": "1930.7",
    "point_article": "26",
    "subpoint_article": "Х"
  },
  {
    "law_id": 1,
    "article": "1930.7",
    "point_article": "26",
    "subpoint_article": "д"
  },
  {
    "law_id": 1,
    "article": "1930.7",
    "point_article": "26",
    "subpoint_article": "21"
  },
  {
    "law_id": 1,
    "article": "1930.7",
    


ТЕСТ 8:
Текст: 'Компания «ТранспортЛайн» была привлечена к ответственности за нарушение норм перевозки грузов. Согла...'

✅ ТЕСТ ПРОЙДЕН
Найдено ссылок: 3
Ожидаемые результаты:
[
  {
    "law_id": 17,
    "article": "929",
    "point_article": null,
    "subpoint_article": null
  },
  {
    "law_id": 17,
    "article": "681.14",
    "point_article": null,
    "subpoint_article": null
  },
  {
    "law_id": 17,
    "article": "1988",
    "point_article": null,
    "subpoint_article": null
  }
]
Полученные результаты:
[
  {
    "law_id": 17,
    "article": "929",
    "point_article": null,
    "subpoint_article": null
  },
  {
    "law_id": 17,
    "article": "681.14",
    "point_article": null,
    "subpoint_article": null
  },
  {
    "law_id": 17,
    "article": "1988",
    "point_article": null,
    "subpoint_article": null
  }
]
--------------------------------------------------------------------------------



ТЕСТ 9:
Текст: 'В целях обеспечения природоохранных мероприятий на территории архипелага были приняты новые меры. Со...'

✅ ТЕСТ ПРОЙДЕН
Найдено ссылок: 3
Ожидаемые результаты:
[
  {
    "law_id": 741,
    "article": "740.8",
    "point_article": null,
    "subpoint_article": null
  },
  {
    "law_id": 741,
    "article": "76.5",
    "point_article": null,
    "subpoint_article": null
  },
  {
    "law_id": 741,
    "article": "292",
    "point_article": null,
    "subpoint_article": null
  }
]
Полученные результаты:
[
  {
    "law_id": 741,
    "article": "740.8",
    "point_article": null,
    "subpoint_article": null
  },
  {
    "law_id": 741,
    "article": "76.5",
    "point_article": null,
    "subpoint_article": null
  },
  {
    "law_id": 741,
    "article": "292",
    "point_article": null,
    "subpoint_article": null
  }
]
--------------------------------------------------------------------------------

ТЕСТ 10:
Текст: 'В ходе аудита было выявлено несоответствие финансово


ТЕСТ 17:
Текст: 'На собрании юридического комитета обсуждались изменения в ст. 1538-14, 891-7 и 693.9-7 ГПК, которые ...'

✅ ТЕСТ ПРОЙДЕН
Найдено ссылок: 8
Ожидаемые результаты:
[
  {
    "law_id": 6,
    "article": "1538-14",
    "point_article": null,
    "subpoint_article": null
  },
  {
    "law_id": 6,
    "article": "891-7",
    "point_article": null,
    "subpoint_article": null
  },
  {
    "law_id": 6,
    "article": "693.9-7",
    "point_article": null,
    "subpoint_article": null
  },
  {
    "law_id": 6,
    "article": "336",
    "point_article": null,
    "subpoint_article": null
  },
  {
    "law_id": 6,
    "article": "1460.11-1",
    "point_article": null,
    "subpoint_article": null
  },
  {
    "law_id": 6,
    "article": "85-4",
    "point_article": null,
    "subpoint_article": null
  },
  {
    "law_id": 6,
    "article": "577",
    "point_article": null,
    "subpoint_article": null
  },
  {
    "law_id": 6,
    "article": "130-12",
    "point_article": null,
 

In [8]:

show_test_details(14)


ТЕСТ 14
Текст:
Главный бухгалтер компании представил отчет о финансовом состоянии, ссылаясь на статьи 1836, 1670.10, 802, 270, 1351 и 930 Положения по ведению бухгалтерского учета и бухгалтерской отчетности в Российской Федерации. Он отметил, что соблюдение этих норм критично для обеспечения прозрачности и достоверности финансовой информации. В связи с этим компания планирует провести внутренний аудит на следующей неделе.

--------------------------------------------------------------------------------

Ожидается найти: 6 ссылок
Найдено: 6 ссылок

ОЖИДАЕМЫЕ РЕЗУЛЬТАТЫ:
1. law_id=1036, article=1836, point=None, subpoint=None
2. law_id=1036, article=1670.10, point=None, subpoint=None
3. law_id=1036, article=802, point=None, subpoint=None
4. law_id=1036, article=270, point=None, subpoint=None
5. law_id=1036, article=1351, point=None, subpoint=None
6. law_id=1036, article=930, point=None, subpoint=None

ПОЛУЧЕННЫЕ РЕЗУЛЬТАТЫ:
1. law_id=1036, article=1836, point=None, subpoint=None
2. law_i